# Model Setup
Model 1 takes in img outputs moving probability
Model 2 takes in a sequence of 16 moving probs outputs falling probability

In [1]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')
import tensorflow_datasets as tfds

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

import cv2

/home/jessica/VIA-Fall-Detection/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jessica/VIA-Fall-Detection/.venv/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/jessica/VIA-Fall-Detection/.venv/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Ten

GPU limitation

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2023-06-01 11:35:30.877396: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-01 11:35:30.914784: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-01 11:35:30.914983: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-01 11:35:30.916119: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

# Building the Models

## Model 1

### Train

In [3]:
data = DataLoader.from_folder(r"./datasets/model1_data")
train_data, test_data = data.split(0.9)

INFO:tensorflow:Load image with size: 2917, num_label: 2, labels: Moving, still.


In [14]:
model = image_classifier.create(train_data, use_augmentation=True)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2_6 (Hub  (None, 1280)             3413024   
 KerasLayerV1V2)                                                 
                                                                 
 dropout_6 (Dropout)         (None, 1280)              0         
                                                                 
 dense_6 (Dense)             (None, 2)                 2562      
                                                                 
Total params: 3,415,586
Trainable params: 2,562
Non-trainable params: 3,413,024
_________________________________________________________________
None
INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use customized resize method bilinear


INFO:tensorflow:Use customized resize method bilinear


Epoch 1/5
82/82 [==============================] - 3s 28ms/step - loss: 0.4442 - accuracy: 0.8266
Epoch 2/5
82/82 [==============================] - 2s 28ms/step - loss: 0.3626 - accuracy: 0.8944
Epoch 3/5
82/82 [==============================] - 2s 28ms/step - loss: 0.3414 - accuracy: 0.9093
Epoch 4/5
82/82 [==============================] - 2s 29ms/step - loss: 0.3384 - accuracy: 0.9135
Epoch 5/5
82/82 [==============================] - 2s 29ms/step - loss: 0.3344 - accuracy: 0.9165


### Evaluate & Export

In [13]:
loss, accuracy = model.evaluate(test_data)
# model.summary()

10/10 [==============================] - 1s 29ms/step - loss: 0.2689 - accuracy: 0.9623


In [ ]:
model.export(export_dir='.')

## Model 2

### Train

In [8]:
default_data = np.loadtxt('./datasets/model2_data/default.txt')
falling_data = np.loadtxt('./datasets/model2_data/falling.txt')

In [33]:
inputs = np.concatenate((default_data, falling_data))
outputs = np.concatenate((np.zeros(len(default_data)), np.ones(len(falling_data))))

# Convert inputs and outputs to TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))
dataset = dataset.shuffle(len(inputs)).batch(128)

In [34]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(16,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(dataset, epochs=6)

Epoch 1/6
5/5 [==============================] - 0s 2ms/step - loss: 0.6934 - accuracy: 0.6962
Epoch 2/6
5/5 [==============================] - 0s 2ms/step - loss: 0.6736 - accuracy: 0.6998
Epoch 3/6
5/5 [==============================] - 0s 1ms/step - loss: 0.6582 - accuracy: 0.7288
Epoch 4/6
5/5 [==============================] - 0s 1ms/step - loss: 0.6426 - accuracy: 0.7722
Epoch 5/6
5/5 [==============================] - 0s 2ms/step - loss: 0.6273 - accuracy: 0.7975
Epoch 6/6
5/5 [==============================] - 0s 2ms/step - loss: 0.6126 - accuracy: 0.8156


### Evaluate & Export

In [12]:
# loss, acc = model.evaluate(X_test, y_test)
# model.summary()
model.save("model2")

INFO:tensorflow:Assets written to: model2/assets


2023-05-31 11:17:06.978328: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
INFO:tensorflow:Assets written to: model2/assets


# Predict

## Reformat Input

In [13]:
def resize_video(input_path, output_path, width, height):
  # Open the video file
  video = cv2.VideoCapture(input_path)

  # Get the original video's width and height
  original_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
  original_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
  
  # Create a VideoWriter object to save the resized video
  fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for the output video
  fps = video.get(cv2.CAP_PROP_FPS)
  writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
  
  while True:
    # Read a frame from the original video
    ret, frame = video.read()
    if not ret: break
    # Resize the frame to the desired width and height
    resized_frame = cv2.resize(frame, (width, height))
    # Write the resized frame to the output video file
    writer.write(resized_frame)

# Release the video capture and writer objects
  video.release()
  writer.release()

In [82]:
# Resize vid
video_path = "s-walk2.mp4"
output_path = "datasets/model1_vids/resized_IMG_0484.MOV"
target_width = 224
target_height = 224

# resize_video(video_path, output_path, target_width, target_height)

# Load resized vid
cap = cv2.VideoCapture(output_path)
# frame_rate = 30
# cap.set(cv2.CAP_PROP_FPS, frame_rate) DOESN'T WORK
print(cap.get(cv2.CAP_PROP_FPS))

30.0


## Pass into model 1

In [83]:
# Initialize list to store the frame classifications
frame_classifications = []
moving_prob = []
# Loop through the frames of the video (need to change to 30 fps)
while True:
    ret, frame = cap.read() 
    #just need to figure out if this is 30 fps

    if not ret: # Break the loop if the video has ended
        break

    ''' Formulate Input Data (frame_rgb) '''
    # Convert the frame to RGB format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Make frame input data and ensure its type matches the model
    frame_rgb = np.expand_dims(frame_rgb, axis=0)

    ''' Classify the Frame '''
    interpreter = tf.lite.Interpreter(model_path="model.tflite")
    interpreter.allocate_tensors()

    # get_output_details() and get_input_details() return list of dictionaries of tensor details
    # keys: name, index, shape, shape_signature, dtype, quantization, ...
    # len(input) = len(output) = 1, so access the first element
    output = interpreter.get_output_details()
    input = interpreter.get_input_details()
    output_index = output[0]['index']
    input_index = input[0]['index']

    # set input -> invoke -> access output
    interpreter.set_tensor(input_index, frame_rgb)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_index)
    # If the output_data shape is (batch_size, num_classes), select the first frame
    output_data = output_data[0]

    # Convert each entry into probability
    output_probs = tf.nn.softmax(output_data.astype(float))

    # Find the index of the highest probability
    predicted_index = np.argmax(output_data)

    # Assuming you have a list of class labels corresponding to the model's output classes
    class_labels = ["Moving", "Still"]

    # Get the predicted class label
    predicted_class = class_labels[predicted_index]

    # Print the predicted class label
    # print("Predicted Class:", predicted_class)
    frame_classifications.append((predicted_class, max(output_probs.numpy())))
    
    prob = np.around(max(output_probs.numpy()), decimals = 2)
    if predicted_class == "Still":
        
        moving_prob.append(np.subtract(1, prob))
    else:
        moving_prob.append(prob)

## Prepare data for model 2

In [84]:
moving_probs_trimmed = moving_prob[:-(len(moving_prob)%16)]
model2_in = np.array(moving_probs_trimmed).reshape((len(moving_prob)//16, 16))
# print(model2_in)

## Pass into Model 2 and get final prediction

In [85]:
model2 = tf.keras.models.load_model("model2")

In [86]:
vid_preds = model2.predict(model2_in)

In [87]:
bools = vid_preds.reshape((1, len(vid_preds))) > 0.5
print(bools)
print(vid_preds.reshape((1, len(vid_preds))))

[[False  True  True  True]]
[[0.37859526 0.69188964 0.50663507 0.5051739 ]]


### Batch predicting multiple vids

In [ ]:
falling_paths = ["./datasets/model1_vids/splitted_for_model2/resized_IMG0480_falling.MOV",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG0484_falling.MOV",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG0485_falling.MOV",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG1614_falling.MOV",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG1615_falling.MOV",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG1616_falling.MOV"]

default_paths = ["./datasets/model1_vids/splitted_for_model2/resized_IMG0480_default.MOV",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG0484_default.MOV",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG0485_default.MOV",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG1614_default.MOV",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG1615_default.MOV",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG1616_default.MOV",
                 "./datasets/model2_vids/resized_jess_jump.MOV",
                 "./datasets/model2_vids/resized_jess_pickup.MOV",
                 "./datasets/model2_vids/resized_jess_run1.MOV",
                 "./datasets/model2_vids/resized_jess_run2.MOV",
                 "./datasets/model2_vids/resized_jess_run3.MOV",
                 "./datasets/model2_vids/resized_jess_run4.MOV",
                 "./datasets/model2_vids/resized_jess_walk1.MOV",
                 "./datasets/model2_vids/resized_jess_walk2.MOV",
                 "./datasets/model2_vids/resized_jess_walk3.MOV",
                 "./datasets/model2_vids/resized_jess_walk4.MOV"]

In [ ]:
def batchPredict(path, probs):
    cap = cv2.VideoCapture(path)
    # Loop through the frames of the video (need to change to 30 fps)
    while True:
        ret, frame = cap.read() 
        if not ret: break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_rgb = np.expand_dims(frame_rgb, axis=0)

        interpreter = tf.lite.Interpreter(model_path="model.tflite")
        interpreter.allocate_tensors()
        output = interpreter.get_output_details()
        input = interpreter.get_input_details()
        output_index = output[0]['index']
        input_index = input[0]['index']

        interpreter.set_tensor(input_index, frame_rgb)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_index)
        output_data = output_data[0]
        output_probs = tf.nn.softmax(output_data.astype(float))

        predicted_index = np.argmax(output_data)
        class_labels = ["Moving", "Still"]
        predicted_class = class_labels[predicted_index]        
        prob = np.around(max(output_probs.numpy()), decimals = 2)

        if predicted_class == "Still": probs.append(np.subtract(1, prob))
        else: probs.append(prob)

In [ ]:
for vid in falling_paths:
    vid_probs = batchPredict(vid)
    print(vid)
    for i in range(len(moving_prob)-15):
        for j in range(i, i+16):
            print(moving_prob[j], end=" ")
        print()
    print()

In [ ]:
for vid in default_paths:
    vid_probs = batchPredict(vid)
    print(vid)
    for i in range(len(moving_prob)-15):
        for j in range(i, i+16):
            print(moving_prob[j], end=" ")
        print()
    print()